In [ ]:
import multiprocessing
multiprocessing.set_start_method('spawn')

In [ ]:
import cv2
import glob
import sys
import numpy
import utils
import os
import time
import argparse
import pathlib
import torch
from torch.autograd import Variable




containing_dir = '.'

fileDir = os.path.dirname(containing_dir)
modelDir = os.path.join(fileDir, 'weights')

parser = argparse.ArgumentParser()

parser.add_argument('--dlibFacePredictor', type=str, help="Path to dlib's face predictor.",
                    default=os.path.join(modelDir, "shape_predictor_68_face_landmarks.dat"))
parser.add_argument('--database', type=str, help='Compare query image to pictures found in [database]',
                    default='~/Pictures/Webcam/')
parser.add_argument('--imgDim', type=int,
                    help="Default image dimension.", default=96)
parser.add_argument('--verbose', action='store_true')
parser.add_argument('--dlib_dim', type=int, help='im size for face recognition', default=224)
parser.add_argument('--query_path', default='', help='query image path')
parser.add_argument('--refresh', type=int, help='Refresh output image [sec]')
parser.add_argument('--webcam', type=int, default=0, help='use webcam')

args = parser.parse_args('')
align = utils.AlignDlib(args.dlibFacePredictor)
prepareOpenFace = utils.prepareOpenFace


In [ ]:
args.verbose = False

In [ ]:

def ReadImage(imgPath):

    if args.verbose:
        print("Processing {}.".format(imgPath))
    bgrImg = cv2.imread(imgPath)
    if args.verbose:
        print("Loaded {}.".format(imgPath))
    
    if bgrImg is None:
        raise Exception("Unable to load image: {}".format(imgPath))
    
    rgbImg = cv2.cvtColor(bgrImg, cv2.COLOR_BGR2RGB)
    if args.verbose:
        print("Converted {}.".format(imgPath))
    
    return rgbImg

def ProcessImage(rgbImg, max_ratio=1, returnBB=False):

    
    start = time.time()
    bb = align.getLargestFaceBoundingBox(rgbImg)
#     while bb is None and max_ratio > ratio:
#         ratio += 0.1
#         rgbImg = cv2.resize(orig_rgbImg, None, fx=ratio, fy=ratio)
#         bb = align.getLargestFaceBoundingBox(rgbImg)

    if args.verbose:
        print("  + Original size: {}".format(rgbImg.shape))

    if bb is None:
        raise RuntimeWarning("Unable to find a face!")
    if args.verbose:
        print("  + Face detection took {} seconds.".format(time.time() - start))
    #bb /= ratio

    alignedFace = align.align(args.imgDim, rgbImg, bb,
                              landmarkIndices=utils.AlignDlib.OUTER_EYES_AND_NOSE)
    if alignedFace is None:
        raise Exception("Unable to align image: {}".format(imgPath))
    if args.verbose:
        print("  + Aligned size: {}".format(alignedFace.shape))
        print("  + Aligned mean: {}".format(alignedFace.mean()))
        print("  + Aligned dev: {}".format(alignedFace.std()))
        print("  + Face alignment took {} seconds.".format(time.time() - start))

    img = numpy.transpose(alignedFace, (2, 0, 1))
    img = img.astype(numpy.float32) / 255.0
    #cv2.imshow('preproc', cv2.cvtColor(alignedFace, cv2.COLOR_BGR2RGB))
    #print(numpy.min(img), numpy.max(img))
    #print(numpy.sum(img[0]), numpy.sum(img[1]), numpy.sum(img[2]))
    I_ = torch.from_numpy(img).unsqueeze(0)
    if useCuda:
        I_ = I_.cuda()
    if returnBB:
        return I_, utils.rect_to_bb(bb, ratio)
    return I_

def find_k_nearest(query_im, k=3):
    q_var = Variable(query_im, requires_grad=False)
    q_f, q_res = model(q_var)
    lin_d = ((f_736-q_res)**2).mean(-1)

    cos_d = torch.mm(f_736, q_res.transpose(0, 1)).squeeze()
    cos_d /= (f_736**2).mean()
    #print(lin_d.shape, cos_d.shape)


    #dist = ((f - q_f)**2).mean(-1)
    dist = lin_d
    ds, idxs = dist.topk(k, largest=False)
    ds = ds.data.cpu()
    idxs = idxs.data.cpu()
    #print(ds)
    #print(idxs)
    #for idx, d in zip(idxs, ds):
    #    print('%30s  distance: %0.4f' % (img_paths[idx].split('/')[-1], d))
        #torch.dot()
    return idxs, ds

def draw_bb(frame, bb):
    x, y, w, h = bb
    cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 200, 200), 2)

def draw_text(frame, bb, text, distance, threshold=0.0100,
              x_offset=0, y_offset=0, font_scale=2, thickness=2):
    x, y = bb[:2]
    color = (0, 200, 0)
    if distance > threshold:
        color = (0, 0, 200)
        text = 'Ismeretlen'

    cv2.putText(frame, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)



if False:
    #
    useCuda = True
    if useCuda:
        assert torch.cuda.is_available()
    else:
        assert False, 'Sorry, .pth file contains CUDA version of the network only.'

    model = prepareOpenFace()
    model.load_state_dict(torch.load(os.path.join(containing_dir, 'weights', 'openface.pth')))
    model = model.eval()


    #img_paths = glob.glob('/home/botoscs/Pictures/office_badges/*.jpg', recursive=True)
    #img_paths += glob.glob('/home/botoscs/Pictures/Webcam/*.jpg', recursive=True)
    img_paths = glob.glob(args.database + '/**/*.jpg', recursive=True)
    imgs = []
    for img_path in img_paths:
        try:
            img = ReadImage(img_path)
            img = ProcessImage(img)
            
            imgs.append(img)
        except RuntimeWarning as w:
            continue

    I_ = torch.cat(imgs, 0)
    I_ = Variable(I_, requires_grad=False)
    start = time.time()
    f, f_736 = model(I_)
    print("  + Forward pass took {} seconds.".format(time.time() - start))


    if len(args.query_path) == 0:
        cap = cv2.VideoCapture(args.webcam)
        acc_idxs = {}

        imlist = []
        for img_path in img_paths:
            imlist.append(cv2.imread(img_path))

        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        while True:
            ret, bgrImg = cap.read()

            try:
                query_im, rect = ProcessImage(bgrImg, -1, returnBB=True)

                idxs, ds = find_k_nearest(query_im, 5)
                text = img_paths[idxs[0]].split('/')[-2]
                draw_bb(bgrImg, rect)
                draw_text(bgrImg, rect, text, ds[0])
                if args.verbose:
                    #print('\x1b[2J')
                    for idx, d in zip(idxs, ds):
                        print('%30s  distance: %0.4f' % (text, d))
                for idx in idxs:
                    if acc_idxs.get(idx) is None:
                        acc_idxs[idx] = 1
                    else:
                        acc_idxs[idx] += 1
            except Exception as e:
                print(e)
                pass

            finally:
                cv2.imshow('frame', bgrImg)
                cv2.waitKey(1)



    else:
        q_path = args.query_path
        query_im = ProcessImage(ReadImage(q_path))
        find_k_nearest(query_im, 3)

    # in OpenFace's sample code, cosine distance is usually used for f (128d).


In [ ]:
useCuda = True
if useCuda:
    assert torch.cuda.is_available()
else:
    assert False, 'Sorry, .pth file contains CUDA version of the network only.'

model = prepareOpenFace()
model.load_state_dict(torch.load(os.path.join(containing_dir, 'weights', 'openface.pth')))
model = model.eval()

In [ ]:
args.database = '/home/csbotos/video/stable_synced/stable_recordings/camdir-0/2018-04-10/2018-04-10.17/'

In [ ]:

img_paths = glob.glob(args.database + '/**/*.jpg', recursive=True)
print('Total images to be loaded: ', len(img_paths))
imgs = []
img_path = img_paths[0]
print(len(imgs), img_path)

img = ReadImage(img_path)

In [ ]:
img_paths[-1000:-999]

In [ ]:
imgs = []
true_paths = []
i = 0
for img_path in img_paths[-2000:]:
    try:
        img = ReadImage(img_path)
        img = ProcessImage(img)

        imgs.append(img)
        true_paths.append(img_path)
        i += 1
        if i % (len(img_paths) // 30) == 0: print('[%6d : %6d]'%(i, len(img_paths)))
    except RuntimeWarning as w:
        continue

In [ ]:
args.verbose = False

In [ ]:
import numpy as np

In [ ]:
torch.cat(imgs).shape

In [ ]:
del
torch.cuda.empty_cache()

In [ ]:
I_ = torch.cat([x for x in imgs if x.shape == img.shape])

I_ = Variable(I_, requires_grad=False)        

In [ ]:

start = time.time()
f, f_736 = model(I_)
print("  + Forward pass took {} seconds.".format(time.time() - start))

In [ ]:
f.shape, f_736.shape

In [ ]:
result = {
    'fnames':true_paths,
    'f_128':f,
    'f_736':f_736
}

In [ ]:
torch.save(result, 'Example2000Database.dict')

In [ ]:
import dlib

detector = dlib.get_frontal_face_detector()

In [ ]:
%%timeit
detector(img, 1)

In [ ]:
%%timeit
detector(img, 2)

In [ ]:
%%timeit
detector(img, 0)

In [ ]:
%%timeit
detector(img)

In [ ]:
print(align.getAllFaceBoundingBoxes(img))
print(detector(img, 0))

In [ ]:
print(align.getLargestFaceBoundingBox(img))

In [ ]:
database = torch.load('Example2000Database.dict')

In [ ]:
1 

# Cool TSNE plots - without labels

In [ ]:

#from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 16, 10
import ipyvolume as ipv

In [ ]:
X736 = database['f_736'].data.cpu().numpy()
X128 = database['f_128'].data.cpu().numpy()
#X_embedded = TSNE(n_components=2).fit_transform()

In [ ]:
X_embedded128 = TSNE(n_components=2, perplexity=20, n_jobs=40, n_iter=1000, cheat_metric=False, metric='cosine').fit_transform(X128[:])

In [ ]:
X_embedded736 = TSNE(n_components=2, perplexity=20, n_jobs=40, n_iter=1000, cheat_metric=False, metric='cosine').fit_transform(X736[:])

In [ ]:
plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
plt.title("Embedded face2vec dim: 736")
plt.show()

In [ ]:
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1])
plt.title("Embedded face2vec dim: 128")
plt.show()

In [ ]:
start = 790
end = 800
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1])
plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='r')
plt.title("Embedded face2vec dim: 128")
plt.show()

In [ ]:
true_paths[-1]

In [ ]:
true_paths.index('/home/csbotos/video/stable_synced/stable_recordings/camdir-0/2018-04-10/2018-04-10.17/2018-04-10.17-45/hasface-2018-04-10.17-45-00.047788.jpg')

In [ ]:
start = 1490
end = 1540

plt.subplot(121)
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
plt.title('Selected instances int the 128 Feature space')
plt.subplot(122)
plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
plt.title('Selected instances int the 736 Feature space')
plt.show()
for idx in range(start, end):
    
    plt.subplot(221)
    plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
    plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
    plt.scatter(X_embedded128[idx, 0], X_embedded128[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 128 Feature space')
    plt.subplot(222)
    plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
    plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
    plt.scatter(X_embedded736[idx, 0], X_embedded736[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 736 Feature space')
    
    plt.subplot(223)
    plt.imshow(ReadImage(true_paths[idx]))
    plt.subplot(224)
    plt.imshow(imgs[idx].cpu()[0].permute(1, 2, 0))
    
    plt.show()

In [ ]:
start = 595
end = 605

plt.subplot(121)
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
plt.title('Selected instances int the 128 Feature space')
plt.subplot(122)
plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
plt.title('Selected instances int the 736 Feature space')
plt.show()
for idx in range(start, end):
    
    plt.subplot(221)
    plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
    plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
    plt.scatter(X_embedded128[idx, 0], X_embedded128[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 128 Feature space')
    plt.subplot(222)
    plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
    plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
    plt.scatter(X_embedded736[idx, 0], X_embedded736[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 736 Feature space')
    
    plt.subplot(223)
    plt.imshow(ReadImage(true_paths[idx]))
    plt.subplot(224)
    plt.imshow(imgs[idx][0].permute(1, 2, 0))
    
    plt.show()

In [ ]:
start = 0
end = 20

plt.subplot(121)
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
plt.title('Selected instances int the 128 Feature space')
plt.subplot(122)
plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
plt.title('Selected instances int the 736 Feature space')
plt.show()
for idx in range(start, end):
    
    plt.subplot(221)
    plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1], )
    plt.scatter(X_embedded128[start:end, 0], X_embedded128[start:end, 1], c='orange')
    plt.scatter(X_embedded128[idx, 0], X_embedded128[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 128 Feature space')
    plt.subplot(222)
    plt.scatter(X_embedded736[:, 0], X_embedded736[:, 1])
    plt.scatter(X_embedded736[start:end, 0], X_embedded736[start:end, 1], c='orange')
    plt.scatter(X_embedded736[idx, 0], X_embedded736[idx, 1], c='r', marker='x', s=200, linewidth=5)
    plt.title('X marks the instance in the 736 Feature space')
    
    plt.subplot(223)
    plt.imshow(ReadImage(true_paths[idx]))
    plt.subplot(224)
    plt.imshow(imgs[idx][0].permute(1, 2, 0))
    
    plt.show()

In [ ]:
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1])
plt.title("Embedded face2vec dim: 128")
plt.show()

In [ ]:
plt.scatter(X_embedded128[:, 0], X_embedded128[:, 1])
plt.title("Embedded face2vec dim: 128")
plt.show()

In [ ]:
X.shape

In [ ]:
X_embedded128_3d = TSNE(n_components=3, perplexity=10).fit_transform(X128[:400])

In [ ]:

ipv.quickscatter(X_embedded128_3d[:, 0], X_embedded128_3d[:, 1], X_embedded128_3d[:, 2], size=1, marker="sphere")